In [1]:
from aocd import get_data, submit
from operator import itemgetter
import numpy as np
from itertools import chain

In [2]:
example_data = "498,4 -> 498,6 -> 496,6\n503,4 -> 502,4 -> 502,9 -> 494,9"
real_data = get_data(day=14, year=2022)

In [3]:
def create_simulation_environment(input, bottom=False):
    rocks = [[list(map(int,coordinate.split(","))) for coordinate in line.split(" -> ")] for line in input.splitlines()]
    points = list(chain(*rocks)) + [[500,0]]
    min_x = min(points, key=itemgetter(1))[1]
    min_y = min(points, key=itemgetter(0))[0]
    max_x = max(points, key=itemgetter(1))[1]
    max_y = max(points, key=itemgetter(0))[0]

    height = max_x-min_x+1+2
    width =  max(height, 500 - min_y) + max(height, max_y - 500) + 4
    offset_y = 500 - max(height, 500 - min_y)
    offset_x = min_x

    grid = np.full((height, width), 0, dtype=int)
    start = np.array([0-offset_x, 500-offset_y])

    for rock in rocks:
        for i in range(len(rock)-1):
            point, point_next = np.array(sorted([rock[i], rock[i+1]]))-[offset_y, offset_x]
            grid[point[1]:point_next[1]+1,point[0]:point_next[0]+1] = 1

    if bottom:
      grid[-1] = 1

    return grid, start

In [4]:
def simulate(grid, start):
    rounds = 0
    while True:
        position = start
        while True:
            moved = False
            for motion in [[1, 0], [1, -1], [1, 1]]:
                test_position = position + motion
                if not (0 <= test_position[0] <= grid.shape[0]-1 and 0 <= test_position[1] <= grid.shape[1]-1):
                    return rounds

                if grid[tuple(test_position)] == 0:
                    position = test_position
                    moved = True
                    break

            if not moved:
                grid[tuple(position)] = 1
                rounds = rounds + 1
                if tuple(position) == tuple(start):
                    return rounds
                break

In [5]:
grid_1,start_1 = create_simulation_environment(real_data, False)
print(f"Number of rounds until sand leaves the grid: {simulate(grid_1, start_1)}")

grid_2,start_2 = create_simulation_environment(real_data, True)
print(f"Number of rounds until sand stacked up: {simulate(grid_2, start_2)}")

Number of rounds until sand leaves the grid: 779
Number of rounds until sand stacked up: 27426
